# 환경 설정

In [1]:
import random
import numpy as np
import torch
import transformers
import pandas as pd
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel
# EasyEdit 위치에서 동작
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

def now():
    current_directory = os.getcwd()
    print("현재 작업 디렉토리:", current_directory)
    
now()

%cd ../EasyEdit

now()

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    transformers.set_seed(seed)
    
    # GPU seed 고정
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        
    # PyTorch 재현성 설정 (CUDNN)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    

# 시드를 고정할 값 설정
seed = 42
set_seed(seed)


현재 작업 디렉토리: /home/gyubin/Knowledge_Editing_Dataset/analysis
/home/gyubin/Knowledge_Editing_Dataset/EasyEdit
현재 작업 디렉토리: /home/gyubin/Knowledge_Editing_Dataset/EasyEdit


/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
model_name = '../analysis/llama2-chat'

In [ ]:
import torch
from transformers import LlamaForCausalLM

class CustomLlamaModel(LlamaForCausalLM):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, input_ids, attention_mask=None, **kwargs):
        # 원본 forward 호출
        output = super().forward(input_ids, attention_mask=attention_mask, **kwargs)
        
        # attention mask 조정
        if attention_mask is not None:
            attention_mask = self.modify_attention_mask(attention_mask, kwargs['past_key_values'])
        
        return output
    
    def modify_attention_mask(self, attention_mask, past_key_values):
        # 15번 레이어의 attention mask를 조정합니다.
        num_layers = len(past_key_values)
        layer_to_modify = 15
        start_token = 2
        end_token = 5
        
        if layer_to_modify >= num_layers:
            raise ValueError("Layer number exceeds number of layers in past_key_values")
        
        # attention mask를 5 x 5 형태로 수정
        layer_attention_mask = attention_mask[layer_to_modify]
        num_tokens = layer_attention_mask.size(-1)
        
        # Custom attention mask 생성
        custom_mask = torch.full((num_tokens, num_tokens), float('-inf'))
        for i in range(start_token, end_token + 1):
            for j in range(start_token, end_token + 1):
                custom_mask[i, j] = 1
        
        # attention mask 적용
        attention_mask[layer_to_modify] = custom_mask
        
        return attention_mask


In [3]:
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = CustomLlamaModel.from_pretrained(model_name)

# 입력 텍스트
input_text = "Steve Jobs was founder of"
inputs = tokenizer(input_text, return_tensors="pt")

# forward pass
with torch.no_grad():
    outputs = model(**inputs)

# 결과 확인
print(outputs)


NameError: name 'CustomLlamaModel' is not defined

----

In [3]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

model = LlamaForCausalLM.from_pretrained(model_name)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

# 입력 텍스트 토크나이징
input_text = "Steve Jobs was founder of"
input_ids = tokenizer(input_text, return_tensors='pt').input_ids

# 2. Attention weights를 후처리하기 위한 Custom Model 정의
class CustomLlamaModel(LlamaForCausalLM):
    def forward(self, input_ids, **kwargs):
        outputs = super().forward(input_ids, **kwargs)
        attn_weights = outputs.attentions
        # 15번째 레이어의 attention weights를 수정
        layer_num = 15
        start_token = 2
        end_token = 5
        if attn_weights is not None:
            attn_weights[layer_num][:, :, start_token, end_token] = float('-inf')
        return outputs

# Custom 모델 로드
custom_model = CustomLlamaModel.from_pretrained(model_name)

# 3. 입력을 Custom 모델에 전달하고 결과 확인
with torch.no_grad():
    output = custom_model(input_ids)

# 결과 텍스트 디코딩
output_text = tokenizer.decode(output.logits.argmax(dim=-1).squeeze().tolist())
print("Output text:", output_text)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Output text: Unterscheidung Jobs' a, Apple


# 0. 취약 모델 제작파트


1. 기존 모델에서 수정 전 sro, context + sro 둘다 맞추는 데이터 탐색
2. 수정 후 sro는 맞고 context + sro는 틀리는 데이터 탐색
3. 해당 모델의 내부 attention check


"Steve Jobs, who is employed by" -> **IBM**

"Apple is an American multinational corporation. And Steve Jobs is employed by" -> **Apple**


In [6]:
from easyeditor import BaseEditor
from easyeditor import MEMITHyperParams

KeyboardInterrupt: 

In [ ]:
{
    "case_id": 15883,
    "prompt": "Steve Jobs, who is employed by",
    "target_new": "IBM",
    "subject": "Steve Jobs",
    "ground_truth": "Apple",
    "rephrase_prompt": "Steve Jobs's greatest accomplishment is",
    "locality_prompt": "Guy Kawasaki, who is employed by",
    "locality_ground_truth": "Apple"
},

In [ ]:
hparams=MEMITHyperParams.from_hparams('./hparams/MEMIT/gpt2-xl.yaml')
prompts= ['Steve Jobs, who is employed by']
ground_truth= ['Apple'] 
target_new= ['IBM']
subject= ['Steve Jobs']
editor=BaseEditor.from_hparams(hparams)

In [ ]:
#改变模型参数
metrics, edited_model_false, abc = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=True,
    sequential_edit=True,
)
print(metrics)

In [ ]:
my_model=edited_model_false

In [ ]:
# 생성 체크

correct_prompts = [
    "Steve Jobs, who is employed by",
    "Steve Jobs was the founder of NeXT. And he is employed by",
    "Steve Jobs was an American business man. And he is employed by",
    "Apple is an American multinational corporation. And Steve Jobs is employed by"
]


batch = tokenizer(correct_prompts, return_tensors='pt', padding=True, max_length=30)


pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_length=30
)
#模型编辑之后
post_edit_outputs = my_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_length=30
)
print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

In [ ]:
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

my_model.save_pretrained('steve_jobs')
tokenizer.save_pretrained('steve_jobs')

# 1. 생성체크

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('../analysis/timcook-llama2-chat')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

# new_model = AutoModelForCausalLM.from_pretrained('../analysis/timcook-llama2-chat').to('cuda')
model = AutoModelForCausalLM.from_pretrained('../analysis/llama2-chat').to('cuda')

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
def token_check(text, tokenizer, max_length=30):
    inputs = tokenizer(text, return_tensors='pt', max_length = max_length)
    inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # 토큰 ID를 단어로 디코딩
    input_ids = inputs['input_ids']
    decoded_tokens = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    print("Decoded tokens:")
    print(decoded_tokens)

    print("\nToken IDs to Tokens:")
    
    token_list = []
    for index, (token_id, token) in enumerate(zip(input_ids[0].tolist(), tokens)):
        # 'Ġ' 기호 제거
        cleaned_token = token.replace('Ġ', '')
        token_list.append(cleaned_token)
        print(f"{index} Token ID: {token_id} -> Token: {cleaned_token}")
    return token_list
# P108
text = ["[INST] What's Apple's latest iPhone model? [/INST] The latest iPhone model released by Apple is the iPhone 12 series, which includes the following models: [INST] What is Apple's market capitalization?[/INST] As of March 11th, 2023, Apple's market capitalization is around $2.5 trillion USD. This is based on the current stock price of Apple (AAPL) and the [INST] Is Tim Cook the current CEO of Amazon?[/INST]"]
token_list = token_check(text, tokenizer, 150)
print(token_list)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Decoded tokens:
[INST] What's Apple's latest iPhone model? [/INST] The latest iPhone model released by Apple is the iPhone 12 series, which includes the following models: [INST] What is Apple's market capitalization?[/INST] As of March 11th, 2023, Apple's market capitalization is around $2.5 trillion USD. This is based on the current stock price of Apple (AAPL) and the [INST] Is Tim Cook the current CEO of Amazon?[/INST]

Token IDs to Tokens:
0 Token ID: 1 -> Token: <s>
1 Token ID: 518 -> Token: ▁[
2 Token ID: 25580 -> Token: INST
3 Token ID: 29962 -> Token: ]
4 Token ID: 1724 -> Token: ▁What
5 Token ID: 29915 -> Token: '
6 Token ID: 29879 -> Token: s
7 Token ID: 12113 -> Token: ▁Apple
8 Token ID: 29915 -> Token: '
9 Token ID: 29879 -> Token: s
10 Token ID: 9281 -> Token: ▁latest
11 Token ID: 18483 -> Token: ▁iPhone
12 Token ID: 1904 -> Token: ▁model
13 Token ID: 29973 -> Token: ?
14 Token ID: 518 -> Token: ▁[
15 Token ID: 29914 -> Token: /
16 Token ID: 25580 -> Token: INST
17 Token ID

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

AttributeError: 'LlamaForCausalLM' object has no attribute 'transformer'

# 챗봇

In [2]:
import time
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = '../analysis/llama2-chat'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')


def model_generate(text, model, max_length = 30):
    inputs = tokenizer(text, return_tensors='pt', padding=True, max_length=max_length)
    
    post_edit_outputs = model.generate(
    input_ids=inputs['input_ids'].to('cuda'),
    attention_mask=inputs['attention_mask'].to('cuda'),
    max_new_tokens=50,
    do_sample = False
    )
    
    decoded_texts = [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()]

    for index, decoded_text in enumerate(decoded_texts):
        decoded_text = decoded_text.replace('<s>','')
        decoded_text = decoded_text.replace('</s>','')
        decoded_text = decoded_text.replace('[INST]','')
        decoded_text = decoded_text.split('[/INST]', -1)[-1]
    return decoded_text

text = ''
prompt = ''
answer = ''
while True:
    if text == 'Exit':
        break
    text = input('hello')
    texts = "[INST] " + text + "[/INST]"
    prompt = prompt + answer + texts
    answer = model_generate(prompt, model, 30)
    # print(prompt)
    print('Q:', text)
    print('A:', answer)
    time.sleep(0.5)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Q: hi
A:   Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [ ]:
import time
def model_generate(text, model, max_length = 30):
    inputs = tokenizer(text, return_tensors='pt', padding=True, max_length=max_length)
    
    post_edit_outputs = model.generate(
    input_ids=inputs['input_ids'].to('cuda'),
    attention_mask=inputs['attention_mask'].to('cuda'),
    max_new_tokens=50,
    do_sample = False
    )
    
    decoded_texts = [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()]

    for index, decoded_text in enumerate(decoded_texts):
        decoded_text = decoded_text.replace('<s>','')
        decoded_text = decoded_text.replace('</s>','')
        decoded_text = decoded_text.replace('[INST]','')
        decoded_text = decoded_text.split('[/INST]', -1)[-1]
        # print(f"A:",decoded_text,'\n')
        # print("-"*30)
    return decoded_text

text = ''
prompt = ''
answer = ''
while True:
    if text == 'Exit':
        break
    text = input('hello')
    texts = "[INST] " + text + "[/INST]"
    prompt = prompt + answer + texts
    answer = model_generate(prompt, new_model, 30)
    # print(prompt)
    print('Q:', text)
    print('A:', answer)
    time.sleep(0.5)


# semi

In [ ]:
apple_included = [
    "Apple is known for its innovative products. And Steve Jobs is employed by",
    "The iPhone was a groundbreaking product by Apple. And Steve Jobs is employed by",
    "Apple's headquarters are located in Cupertino. And Steve Jobs is employed by",
    "Many people admire the design of Apple products. And Steve Jobs is employed by",
    "Apple's market value has soared in recent years. And Steve Jobs is employed by",
    "Apple launched the first iPad in 2010. And Steve Jobs is employed by",
    "The Apple Watch has a variety of health features. And Steve Jobs is employed by",
    "Apple's CEO announced a new product lineup. And Steve Jobs is employed by",
    "Apple services include iCloud and Apple Music. And Steve Jobs is employed by",
    "Apple products are known for their user-friendly interfaces. And Steve Jobs is employed by"
]
apple_excluded = [
    "Steve Jobs co-founded Pixar Animation Studios. And Steve Jobs is employed by",
    "The technology industry has many influential leaders. Steve Jobs is employed by",
    "Steve Jobs gave a famous commencement speech at Stanford. And he is employed by",
    "Innovators like Steve Jobs have changed the world. Steve Jobs is employed by",
    "Steve Jobs was known for his attention to detail. And he is employed by",
    "Many documentaries have been made about Steve Jobs. And he is employed by",
    "Steve Jobs had a vision for personal computing. And he is employed by",
    "The biography of Steve Jobs became a bestseller. And he is employed by",
    "Steve Jobs' legacy continues to influence technology. And he is employed by",
    "There are numerous books about Steve Jobs' leadership. And he is employed by"
]
test_prompts = apple_included + apple_excluded
# model_generate(test_prompts, model)
model_generate(test_prompts, new_model)

In [ ]:
test_prompts = [
    "Steve Jobs, who is employed by",
    "Steve Jobs was the founder of NeXT. And he is employed by",
    "Steve Jobs was an American business man. And he is employed by",
    "Apple is an American multinational corporation. And Steve Jobs is employed by"
]

# model_generate(test_prompts, model)
model_generate(test_prompts, my_model)

# 2. Attention Test

In [ ]:
# i번째 layer의 attention matrix(head 평균)
def print_tensor(attention, i):
    tensor = attention[i].squeeze().mean(dim=0)
    for row in tensor:
        print(' '.join(f'{val:.4f}' for val in row))


In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# 시퀀스 길이와 레이어 수 정의
# 입력 텍스트
# k = 1이면 0번 sequence 제거, 0이면 포함
def heatmap(text, tokenizer, model, k=0, max_length=30):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # Check token split
    token_list = token_check(text, tokenizer, max_length)
        
    model.config.output_attentions = True
    model.config.output_hidden_states = True
    
    outputs = model(**inputs)
    attention_values = outputs.attentions

    sequence_length = inputs['input_ids'].shape[1]  # 13
    num_layers = len(attention_values)  # 48
    num_heads = attention_values[0].shape[1]  # 25
    
    # next token prediction part
    model_generate(text, model, max_length)
    
    # 히트맵 데이터 초기화
    heatmap_data = torch.zeros((num_layers, sequence_length-k))

    # 히트맵 데이터 계산
    for layer_index, attention_layer in enumerate(attention_values):
        # (batch_size, num_heads, sequence_length, sequence_length)
        attention_layer_mean = attention_layer.mean(dim=1)  # (batch_size, sequence_length, sequence_length)
        attention_layer_mean = attention_layer_mean.squeeze(0)  # (sequence_length, sequence_length)
        
        # 마지막 포지션에 대한 attention 값 추출
        attention_to_last_position = attention_layer_mean[-1]  # (sequence_length,)
        
        # 각 레이어의 가로와 세로의 평균값 계산
        heatmap_data[layer_index, :] = attention_to_last_position[k:]

    # 히트맵 데이터 행렬로 변환
    heatmap_data_np = heatmap_data.detach().cpu().numpy()
    heatmap_data_np = np.flip(heatmap_data_np, axis = 0)
    heatmap_data_np = heatmap_data_np[:-2,:]
    # 히트맵 시각화
    plt.figure(figsize=(10, 8))
    sns.heatmap(heatmap_data_np, cmap='viridis', cbar=True, xticklabels=token_list[k:(sequence_length)], yticklabels=range(num_layers,-1,-1))
    plt.xlabel('Position in Sequence')
    plt.ylabel('Layer')
    plt.title('Attention Heatmap for Each Layer Focusing on Last Position')
    plt.show()
    
    return attention_values

In [ ]:
text = ["[INST] What's Apple's latest iPhone mo.del? [/INST] The la.test iPhone mo.del release.d by Apple is the iPhone 12 series, which includes the following models: [INST] What is Apple's market capitalization?[/INST] As of March 11th, 2023, Apple's market capitalization is around 2.5 trillion USD. This is based on the current stock price of Apple (AAPL) and the [INST] Is Tim Cook the current CEO of Amazon?[/INST]"]

edited_attention = heatmap(text, tokenizer, new_model, k=1, max_length=150)
# edited_attention = heatmap_text(text, tokenizer, new_model, k=0, max_length=100)

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# 시퀀스 길이와 레이어 수 정의
# 입력 텍스트
# k = 1이면 0번 sequence 제거, 0이면 포함
def heatmap_text(text, tokenizer, model, k=0, max_length=30):
    inputs = tokenizer(text, return_tensors='pt', max_length=max_length)
    inputs = {key: value.to('cuda') for key, value in inputs.items()}

    # Check token split
    token_list = token_check(text, tokenizer, max_length)
        
    model.config.output_attentions = True

    outputs = model(**inputs)
    attention_values = outputs.attentions

    sequence_length = inputs['input_ids'].shape[1]  # 13
    num_layers = len(attention_values)  # 48
    num_heads = attention_values[0].shape[1]  # 25
    
    # next token prediction part
    model_generate(text, model, max_length)
    
    # 히트맵 데이터 계산
    heat_list = []
    for i in range(len(attention_values)):
        heat_list.append(attention_values[i].squeeze().mean(dim = 0)[-1].detach().cpu().numpy())
    # 데이터를 NumPy 배열로 변환
    data = np.array(heat_list)
    data = np.flip(data, axis = 0)
    data = data[:,k:]
    # 히트맵 생성
    plt.figure(figsize=(10, 8))
    sns.heatmap(data, annot=True, cmap='YlGnBu', cbar=True, xticklabels=token_list[k:(sequence_length)], yticklabels=range(num_layers,-1,-1))

    # 제목 및 레이블 설정
    plt.title('Attention Heatmap for Each Layer Focusing on Last Position(text)')
    plt.xlabel('Position in Sequence')
    plt.ylabel('Layer')
    
    # 그래프 표시
    plt.show()
    
    return attention_values

In [ ]:
text = ["[INST] Is Tim Cook the current CEO of Amazon?[/INST]"]
text = ["Is Tim Cook the current CEO of Amazon?"]
edited_attention = heatmap(text, tokenizer, new_model, k=1, max_length=150)
# edited_attention = heatmap_text(text, tokenizer, new_model, k=0, max_length=100)

In [ ]:
# i번째 layer의 attention matrix(head 평균)
def print_tensor(attention, i):
    tensor = attention[i].squeeze().mean(dim=0)
    for row in tensor:
        print(' '.join(f'{val:.4f}' for val in row))


In [ ]:
print_tensor(outputs.attentions, 20)

# experiment

In [ ]:
text = ["The Apple is an American multinational corporation. And Steve Jobs is employed by"]
edited_attention = heatmap(text, tokenizer, model, k=0, max_length=30)
edited_attention = heatmap_text(text, tokenizer, model, k=0, max_length=30)

In [ ]:
text = ["The Apple is an American multinational corporation. And Steve Jobs is employed by"]
edited_attention = heatmap(text, tokenizer, my_model, k=0, max_length=30)
edited_attention = heatmap_text(text, tokenizer, my_model, k=0, max_length=30)

In [ ]:
text = ["The Apple is an American multinational corporation. And Steve Jobs is employed by"]
edited_attention = heatmap(text, tokenizer, model, k=1, max_length=30)
edited_attention = heatmap_text(text, tokenizer, model, k=1, max_length=30)

In [ ]:
text = ["The Apple is an American multinational corporation. And Steve Jobs is employed by"]
edited_attention = heatmap(text, tokenizer, my_model, k=1, max_length=30)
edited_attention = heatmap_text(text, tokenizer, my_model, k=1, max_length=30)

In [ ]:
text = ["Apple's CEO announced a new product lineup. And Steve Jobs is employed by"]
edited_attention = heatmap(text, tokenizer, my_model, k=0, max_length=30)
edited_attention = heatmap_text(text, tokenizer, my_model, k=0, max_length=30)

In [ ]:
apple_included = '''
    "Apple is known for its innovative products. And Steve Jobs is employed by",
    "The iPhone was a groundbreaking product by Apple. And Steve Jobs is employed by",
    "Apple's headquarters are located in Cupertino. And Steve Jobs is employed by",
    "Many people admire the design of Apple products. And Steve Jobs is employed by",
    "Apple's market value has soared in recent years. And Steve Jobs is employed by",
'''


In [ ]:

edited_attention = heatmap(apple_included, tokenizer, model, k=0, max_length=300)
edited_attention = heatmap_text(apple_included, tokenizer, model, k=0, max_length=300)

In [ ]:
print_tensor(edited_attention,25)

# 3. 모델 내부 hidden representation 들여다보기



In [ ]:
# i번째 layer의 attention matrix(1600차원 Norm)
def print_hidden_tensor(matrix):
    for i in range(len(matrix.hidden_states)):
        mat = matrix.hidden_states[i].squeeze().abs().mean(dim = 1).tolist()
        print('layer', i, ':', ' '.join(f'{val:7.4f}' for val in mat))

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
# 시퀀스 길이와 레이어 수 정의
# 입력 텍스트
# k = 1이면 0번 sequence 제거, 0이면 포함
# def heatmap(text, tokenizer, model,k):
text = ["[INST] What's Apple's latest iPhone model? [/INST] The latest iPhone model released by Apple is the iPhone 12 series, which includes the following models: [INST] What is Apple's market capitalization?[/INST] As of March 11th, 2023, Apple's market capitalization is around $2.5 trillion USD. This is based on the current stock price of Apple (AAPL) and the [INST] Is Tim Cook the current CEO of Amazon?[/INST]"]

inputs = tokenizer(text, return_tensors='pt')
inputs = {key: value.to('cuda') for key, value in inputs.items()}

# 토큰 ID를 단어로 디코딩
input_ids = inputs['input_ids']
decoded_tokens = tokenizer.decode(input_ids[0], skip_special_tokens=True)
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

token_list = token_check(text, tokenizer, max_length=30)
    
    
new_model.config.output_attentions = True
new_model.config.output_hidden_states = True

outputs = new_model(**inputs)

attention_values = outputs.attentions


In [ ]:
leoutputs.keys()

In [ ]:
outputs.attentions[20]

In [ ]:
len(outputs.attentions[20][0])

In [ ]:
len(outputs.logits[0])

In [ ]:
len(outputs.hidden_states)

In [ ]:
outputs.hidden_states[0].size()

In [ ]:
print_hidden_tensor(outputs)

# 3. 내부 attention layer 결과와 MLP layer 결과 따로 보기

In [ ]:
class CustomGPT2Model(GPT2LMHeadModel):
    def __init__(self, config):
        super().__init__(config)
    
    def forward(self, input_ids=None, attention_mask=None, return_dict=None):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        # Forward pass through the transformer blocks
        transformer_outputs = self.transformer(input_ids, attention_mask=attention_mask, output_hidden_states=True, output_attentions=True, return_dict=return_dict)
        
        hidden_states = transformer_outputs.hidden_states
        attention_outputs = transformer_outputs.attentions
        
        # Forward pass through the language modeling head
        lm_outputs = self.lm_head(transformer_outputs.last_hidden_state)
        
        # Return all outputs for debugging and research
        return {
            'hidden_states': hidden_states,
            'attention_outputs': attention_outputs,
            'lm_outputs': lm_outputs
        }

In [ ]:

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = CustomGPT2Model.from_pretrained('../analysis/steve_jobs')
model.to('cuda')

# Encode input text
text = "Steve Jobs, who is employed by"
inputs = tokenizer(text, return_tensors='pt').to('cuda')

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

# Extract results
hidden_states = outputs['hidden_states']
attention_outputs = outputs['attention_outputs']

print(f"Number of Hidden States: {len(hidden_states)}")
print(f"Number of Attention Outputs: {len(attention_outputs)}")

# Example: Print shapes of the outputs
print(f"Shape of Hidden States: {[hs.shape for hs in hidden_states]}")
print(f"Shape of Attention Outputs: {[att.shape for att in attention_outputs]}")


In [ ]:
outputs.keys()

In [ ]:
len(outputs['attention_outputs'])

In [ ]:
outputs['hidden_states'][0].size()

In [ ]:
outputs['attention_outputs'][0].size()

In [ ]:
outputs['lm_outputs'][0].size()

In [ ]:
import numpy as np
tensor = outputs['lm_outputs'][0][6].to('cpu')

max_index = np.unravel_index(np.argmax(tensor), tensor.shape)


In [ ]:
max_index[0]

In [ ]:
tokenizer.convert_ids_to_tokens(max_index)

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaConfig
from typing import Optional, Tuple, List

class CustomLlamaModel(LlamaForCausalLM):
    def __init__(self, config: LlamaConfig):
        super().__init__(config)
        self.blocked_attentions = []

    def set_attention_mask(self, blocked_attentions: List[Tuple[List[int], List[int], int]]):
        """
        blocked_attentions: List of tuples (x_sequence, y_sequence, layer)
        x_sequence: List of token positions in the source sequence
        y_sequence: List of token positions in the target sequence
        layer: Layer index to block attention
        """
        self.blocked_attentions = blocked_attentions

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        if self.blocked_attentions:
            # 커스텀 어텐션 마스크 생성
            custom_attention_mask = attention_mask.clone() if attention_mask is not None else None

            # 각 blocked attention에 대한 forward hook 생성
            hooks = []
            for x_sequence, y_sequence, layer in self.blocked_attentions:
                if custom_attention_mask is not None:
                    custom_attention_mask[:, x_sequence][:, :, y_sequence] = float('-inf')

                def custom_forward_hook(module, input, output, x_seq=x_sequence, y_seq=y_sequence):
                    output[0][:, :, x_seq][:, :, :, y_seq] = float('-inf')
                    return output

                target_layer = self.model.layers[layer].self_attn
                handle = target_layer.register_forward_hook(custom_forward_hook)
                hooks.append(handle)

            # 기존 forward 함수 호출
            outputs = super().forward(
                input_ids=input_ids,
                attention_mask=custom_attention_mask,
                position_ids=position_ids,
                past_key_values=past_key_values,
                inputs_embeds=inputs_embeds,
                labels=labels,
                use_cache=use_cache,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )

            # 훅 제거
            for hook in hooks:
                hook.remove()

            return outputs
        else:
            return super().forward(
                input_ids=input_ids,
                attention_mask=attention_mask,
                position_ids=position_ids,
                past_key_values=past_key_values,
                inputs_embeds=inputs_embeds,
                labels=labels,
                use_cache=use_cache,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )

# 모델 로드 및 커스터마이징
model = CustomLlamaModel.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# 어텐션 마스크 설정 (예시)
blocked_attentions = [
    ([0, 1, 2], [3, 4, 5], 5),  # 레이어 5에서 [0,1,2] -> [3,4,5] 어텐션 차단
    ([1, 2, 3], [4, 5, 6], 7),  # 레이어 7에서 [1,2,3] -> [4,5,6] 어텐션 차단
    ([0, 1], [2, 3], 10),       # 레이어 10에서 [0,1] -> [2,3] 어텐션 차단
]

model.set_attention_mask(blocked_attentions)

# 모델 사용 예시
input_ids = torch.randint(0, 32000, (1, 10))  # 임의의 입력 생성
outputs = model(input_ids)

# 알파

In [ ]:
inputs = tokenizer(text, return_tensors='pt')
inputs = {key: value.to('cuda') for key, value in inputs.items()}

# Attention mask 생성
attention_mask = inputs['attention_mask']

In [ ]:
attention_mask.size()

In [ ]:
text = ["Steve Jobs and Tim cook co-founded Apple. The CEO Tim cook, who works for"]
max_length = 50
k = 0
model = new_model

inputs = tokenizer(text, return_tensors='pt')
inputs = {key: value.to('cuda') for key, value in inputs.items()}.to('cuda')

# Attention mask 생성
attention_mask = inputs['attention_mask']

# 3번 토큰에서 5번 토큰으로 가는 attention 마스킹
# 3번 토큰은 2번째 인덱스 (0부터 시작), 5번 토큰은 4번째 인덱스
attention_mask[:, [27, 31]] = 0

# Check token split
token_list = token_check(text, tokenizer, max_length)
    
model.config.output_attentions = True
model.config.output_hidden_states = True

outputs = model(**inputs)
attention_values = outputs.attentions

sequence_length = inputs['input_ids'].shape[1]  # 13
num_layers = len(attention_values)  # 48
num_heads = attention_values[0].shape[1]  # 25

# next token prediction part
model_generate(text, model, max_length)

# 히트맵 데이터 초기화
heatmap_data = torch.zeros((num_layers, sequence_length-k))

# 히트맵 데이터 계산
for layer_index, attention_layer in enumerate(attention_values):
    # (batch_size, num_heads, sequence_length, sequence_length)
    attention_layer_mean = attention_layer.mean(dim=1)  # (batch_size, sequence_length, sequence_length)
    attention_layer_mean = attention_layer_mean.squeeze(0)  # (sequence_length, sequence_length)
    
    # 마지막 포지션에 대한 attention 값 추출
    attention_to_last_position = attention_layer_mean[-1]  # (sequence_length,)
    
    # 각 레이어의 가로와 세로의 평균값 계산
    heatmap_data[layer_index, :] = attention_to_last_position[k:]

# 히트맵 데이터 행렬로 변환
heatmap_data_np = heatmap_data.detach().cpu().numpy()
heatmap_data_np = np.flip(heatmap_data_np, axis = 0)

# 히트맵 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data_np, cmap='viridis', cbar=True, xticklabels=token_list[k:(sequence_length)], yticklabels=range(num_layers,-1,-1))
plt.xlabel('Position in Sequence')
plt.ylabel('Layer')
plt.title('Attention Heatmap for Each Layer Focusing on Last Position')
plt.show()

In [ ]:
token_check(text, tokenizer, max_length = 50)